In [2]:
%pip install pandas 
%pip install numpy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np



In [4]:
stavbe2024 = pd.read_csv(r"clean_GURS\new_prodaja2024.csv", sep=',', encoding='utf-8')
stavbe2024.head(10)

,ime_ko,naselje,letnica_izgradnje,m2_bruto,m2_neto,LETO,cena,novogradnja
0,TABOR,LJUBLJANA,1895,120.7,82.7,2024,355000,NaN
1,STOŽICE,LJUBLJANA,2011,97.4,94.3,2024,515000,NaN
2,STOŽICE,LJUBLJANA,2001,164.4,132.6,2024,365000,NaN
3,ZGORNJA ŠIŠKA,LJUBLJANA,2023,129.8,88.3,2024,358419,1.0
4,ČRNUČE,LJUBLJANA,2023,84.0,73.2,2024,261163,1.0
5,ČRNUČE,LJUBLJANA,2023,126.8,109.6,2024,398070,1.0
6,MOSTE,LJUBLJANA,1958,70.7,61.2,2024,258000,NaN
7,BRINJE I,LJUBLJANA,1968,129.4,84.6,2024,400000,NaN
8,VIČ,LJUBLJANA,1965,60.5,51.1,2024,250000,NaN
9,SLAPE,LJUBLJANA,1973,39.7,35.5,2024,171000,NaN


In [5]:
stavbe2024.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2556 entries, 0 to 2555
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ime_ko             2556 non-null   object 
 1   naselje            2546 non-null   object 
 2   letnica_izgradnje  2556 non-null   int64  
 3   m2_bruto           2556 non-null   float64
 4   m2_neto            2556 non-null   float64
 5   LETO               2556 non-null   int64  
 6   cena               2556 non-null   int64  
 7   novogradnja        911 non-null    float64
dtypes: float64(3), int64(3), object(2)
memory usage: 159.9+ KB


In [6]:
#preparing the year column for Power BI
stavbe2024['LETO'] = pd.to_datetime("2024-01-01")

In [7]:

# creating a boolean column 'NOVOGRADNJA_1' based on 'NOVOGRADNJA'
stavbe2024['novogradnja_1'] = stavbe2024['novogradnja'] == 1

# Preview the new column
stavbe2024[['novogradnja', 'novogradnja_1']].sample(10)


,novogradnja,novogradnja_1
344,NaN,False
1875,1.0,True
2524,NaN,False
763,NaN,False
2188,NaN,False
1705,NaN,False
1845,NaN,False
689,NaN,False
1931,NaN,False
89,NaN,False


In [8]:
stavbe2024[stavbe2024['m2_neto'] == 0].count()

ime_ko               10
naselje               6
letnica_izgradnje    10
m2_bruto             10
m2_neto              10
LETO                 10
cena                 10
novogradnja           3
novogradnja_1        10
dtype: int64

In [10]:
stavbe2024[stavbe2024['m2_neto'].isna()]

,ime_ko,naselje,letnica_izgradnje,m2_bruto,m2_neto,LETO,cena,novogradnja,novogradnja_1


In [11]:
stavbe2024.loc[stavbe2024['m2_neto'] == 0, 'm2_neto'] = np.nan
stavbe2024.fillna({'m2_neto': stavbe2024['m2_bruto']}, inplace=True)

In [ ]:
# is there a house/flet with 0 m2_bruto?
stavbe2024[stavbe2024['m2_bruto'] == 0]

,ime_ko,naselje,letnica_izgradnje,m2_bruto,m2_neto,LETO,cena,novogradnja,novogradnja_1
1834,VOLAVLJE,ZGORNJA BESNICA,1989,0.0,0.0,2024-01-01,33000,NaN,False


In [13]:
stavbe2024.drop(index=1834, inplace=True)  # Dropping the row with index 1834

In [14]:
def oceni_sobnost(m2):
    if pd.isna(m2):
        return None
    if m2 < 35:
        return 1
    elif m2 < 50:
        return 2
    elif m2 < 70:
        return 3
    elif m2 < 90:
        return 4
    elif m2 < 110:
        return 5
    else:
        return 6

In [15]:
stavbe2024['sobnost_ocenjena'] = stavbe2024['m2_neto'].apply(oceni_sobnost)

In [16]:
stavbe2024.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2555 entries, 0 to 2555
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   ime_ko             2555 non-null   object        
 1   naselje            2545 non-null   object        
 2   letnica_izgradnje  2555 non-null   int64         
 3   m2_bruto           2555 non-null   float64       
 4   m2_neto            2555 non-null   float64       
 5   LETO               2555 non-null   datetime64[ns]
 6   cena               2555 non-null   int64         
 7   novogradnja        911 non-null    float64       
 8   novogradnja_1      2555 non-null   bool          
 9   sobnost_ocenjena   2555 non-null   int64         
dtypes: bool(1), datetime64[ns](1), float64(3), int64(3), object(2)
memory usage: 202.1+ KB


In [17]:
# Droping the original 'NOVOGRADNJA' column, as we now have 'NOVOGRADNJA_1' which has the same information in a boolean format.
stavbe2024.drop(columns=['novogradnja'], inplace=True)

In [18]:
# Uniforming the column names to lowercase
stavbe2024.columns = [col.lower() for col in stavbe2024.columns]

In [19]:
#uniforming the all values in all the columns to lowercase
for col in stavbe2024.columns:
    if stavbe2024[col].dtype == 'object':
        stavbe2024[col] = stavbe2024[col].str.lower()

In [20]:
stavbe2024['ime_ko'].unique()

array(['tabor', 'stožice', 'zgornja šiška', 'črnuče', 'moste', 'brinje i',
       'vič', 'slape', 'dravlje', 'udmat', 'spodnja šiška', 'kašelj',
       'štepanja vas', 'vižmarje', 'ježica', 'trnovsko predmestje',
       'bežigrad', 'stanežiče', 'ajdovščina', 'glince', 'šentpeter',
       'poljansko predmestje', 'nove jarše', 'zelena jama',
       'šentvid nad ljubljano', 'šmartno ob savi', 'trebeljevo',
       'gradišče i', 'ljubljana mesto', 'gradišče ii', 'brdo',
       'zadobrova', 'karlovško predmestje', 'tacen', 'šujica', 'bizovik',
       'podgorica', 'dobrova', 'dobrunje', 'krakovsko predmestje',
       'nadgorica', 'brinje ii', 'rudnik', 'šmartno pod šmarno goro',
       'sostro', 'prule', 'gameljne', 'golovec', 'volavlje', 'podsmreka',
       'tomišelj', 'grič'], dtype=object)

In [16]:
# checking all the empty values in the 'naselje' column
stavbe2024[stavbe2024['naselje'].isna()]

,ime_ko,naselje,letnica_izgradnje,m2_bruto,m2_neto,leto,cena,novogradnja_1,sobnost_ocenjena
23,kašelj,NaN,1970,69.7,51.3,2024-01-01,255000,False,3
56,šentpeter,NaN,1950,18.0,18.0,2024-01-01,215500,False,1
665,šentpeter,NaN,1950,18.0,18.0,2024-01-01,215500,False,1
1216,bizovik,NaN,2023,159.7,0.0,2024-01-01,200000,False,1
2084,glince,NaN,1969,81.9,64.7,2024-01-01,208000,False,3
2135,bežigrad,NaN,1972,390.3,129.3,2024-01-01,450000,False,6
2285,bizovik,NaN,2023,172.0,0.0,2024-01-01,395000,True,1
2390,šentpeter,NaN,1850,149.6,143.6,2024-01-01,264000,False,6
2441,vižmarje,NaN,2023,114.8,0.0,2024-01-01,420000,True,1
2497,bizovik,NaN,2023,172.0,0.0,2024-01-01,395000,True,1


In [21]:
# changing all the empty values in the 'naselje' column to 'ljubljana'
stavbe2024.fillna({'naselje': 'ljubljana'}, inplace=True)

In [22]:
stavbe2024.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2555 entries, 0 to 2555
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   ime_ko             2555 non-null   object        
 1   naselje            2555 non-null   object        
 2   letnica_izgradnje  2555 non-null   int64         
 3   m2_bruto           2555 non-null   float64       
 4   m2_neto            2555 non-null   float64       
 5   leto               2555 non-null   datetime64[ns]
 6   cena               2555 non-null   int64         
 7   novogradnja_1      2555 non-null   bool          
 8   sobnost_ocenjena   2555 non-null   int64         
dtypes: bool(1), datetime64[ns](1), float64(2), int64(3), object(2)
memory usage: 182.1+ KB


In [23]:
stavbe2024.to_csv(r"clean_GURS\stavbe2024_cleaned.csv", index=False, encoding='utf-8')